In [1]:
# import
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import os

# config
data_path = '/home/chc/data'

# Question 4

In [2]:
# Download data
question4_data_path = data_path + '/text-similarity'
question4_data_file = question4_data_path +'/test.csv'

if not os.path.exists(question4_data_path):
    os.mkdir(question4_data_path)

if not os.path.exists(question4_data_file):
    print('Downloading data as '+question4_data_file)
    os.system('cd '+question4_data_path+';kg dataset -o rishisankineni -d text-similarity')
else:
    print('Data file exist '+question4_data_file)

Data file exist /home/chc/data/text-similarity/test.csv


b) Load the data to a Spark/Pandas data frame

In [3]:
# Load data up
question4_data = pd.read_csv(question4_data_file)
question4_data.head()

,test_id,description_x,description_y,same_security
0,0,semtech corp,semtech corporation,NaN
1,1,vanguard mid cap index,vanguard midcap index - a,NaN
2,2,spdr gold trust gold shares,spdr gold trust spdr gold shares,NaN
3,3,vanguard total bond index adm,vanguard total bond market index,NaN
4,4,oakmark international fund class i,oakmark international cl i,NaN


c) Calculate similarity between description_x and description_y and store resultant
scores in a new column

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

vect = CountVectorizer(stop_words=stopwords)
vocabulary =  vect.fit(question4_data['description_x'] + question4_data['description_y']).vocabulary_
tfidf_vect = TfidfVectorizer(stop_words=stopwords,
                             vocabulary=vocabulary)
x_mat = tfidf_vect.fit_transform(question4_data['description_x'])
y_mat = tfidf_vect.fit_transform(question4_data['description_y'])
cosimilarities = cosine_similarity(x_mat, y_mat)

print(cosimilarities.shape)

# We only intereted similarity between description_x vs description_y by row
scores = []
for i in range(len(cosimilarities)):
    scores.append(cosimilarities[i][i])
    
question4_data['score_withoutspark'] = scores
question4_data.head(25)

[nltk_data] Downloading package stopwords to /home/chc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(516, 516)


,test_id,description_x,description_y,same_security,score_withoutspark
0,0,semtech corp,semtech corporation,NaN,0.680144
1,1,vanguard mid cap index,vanguard midcap index - a,NaN,0.315582
2,2,spdr gold trust gold shares,spdr gold trust spdr gold shares,NaN,0.953975
3,3,vanguard total bond index adm,vanguard total bond market index,NaN,0.723185
4,4,oakmark international fund class i,oakmark international cl i,NaN,0.761279
5,5,pfizer inc div: 1.200,pfizer inc com,NaN,0.606349
6,6,spartan global ex us index fid adv cl,sptn glb xus idx adv,NaN,0.144193
7,7,vanguard total bond market idx-adm,vanguard total bond market index fund investor...,NaN,0.510525
8,8,banco latinoamericano de exportacio class e co...,banco latinoamericano come-e,NaN,0.462037
9,9,baidu inc fadr 1 adr reps 0.1 ord shs,baidu inc spons ads repr 0.10 ord cls a us0.00005,NaN,0.268183


d) Parallelise the matching process using SPARK environment.

In [5]:
# Init spark
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="SeekSpark")
sc.setLogLevel('DEBUG')